# Given the three Datasets, anazlye the datasets, and set up a structure for ML

In [1]:
!pip install geopy

## Importing modules

In [2]:
import pandas as pd
import sqlite3
import matplotlib
import sys
import datetime
import matplotlib as plt
import geopy

## Database Set Up

In [3]:
db = sqlite3.connect('bike_database.db')

# Easy Query function
def run_query(query):
    return pd.read_sql_query(query, db)

## Preliminary Data Analysis

In [4]:
# Data frames
station_data = pd.read_csv("station_data.csv")
trip_data = pd.read_csv("trip_data.csv")
weather_data = pd.read_csv("weather_data.csv")

In [5]:
station_data.head()

,Id,Name,Lat,Long,Dock Count,City
0,2,San Jose Diridon Caltrain Station,37.329732,-121.901782,27,San Jose
1,3,San Jose Civic Center,37.330698,-121.888979,15,San Jose
2,4,Santa Clara at Almaden,37.333988,-121.894902,11,San Jose
3,5,Adobe on Almaden,37.331415,-121.893200,19,San Jose
4,6,San Pedro Square,37.336721,-121.894074,15,San Jose


### Notes
 1. Id is the primary key
 2. To get a relationship between dock place and zip code we need to add a zipcode to this field, based of unique lat and long combos

In [6]:
trip_data.head()

,TripID,Start Date,Start Station,End Date,End Station,Subscriber Type,Duration
0,913460,31/08/2015 23:26,50,31/08/2015 23:39,70,Subscriber,NaN
1,913459,31/08/2015 23:11,31,31/08/2015 23:28,27,Subscriber,NaN
2,913455,31/08/2015 23:13,47,31/08/2015 23:18,64,Subscriber,NaN
3,913454,31/08/2015 23:10,10,31/08/2015 23:17,8,Subscriber,NaN
4,913453,31/08/2015 23:09,51,31/08/2015 23:22,60,Customer,NaN


### Notes
1. TripID is primary key
2. Start Station is a forgien key as is End Station in station db
    -and-
   Start Date and date in weather db
3. It'd be useful to have a duration field in this table so I dont have to calculate its everytime.

In [7]:
weather_data.head()

,Date,Max TemperatureF,Mean TemperatureF,Min TemperatureF,Max Dew PointF,MeanDew PointF,Min DewpointF,Max Humidity,Mean Humidity,Min Humidity,...,Mean VisibilityMiles,Min VisibilityMiles,Max Wind SpeedMPH,Mean Wind SpeedMPH,Max Gust SpeedMPH,PrecipitationIn,CloudCover,Events,WindDirDegrees,Zip
0,01/09/2014,83.0,70.0,57.0,58.0,56.0,52.0,86.0,64.0,42.0,...,10.0,8.0,16.0,7.0,20.0,0.0,0.0,NaN,290.0,94107
1,02/09/2014,72.0,66.0,60.0,58.0,57.0,55.0,84.0,73.0,61.0,...,10.0,7.0,21.0,8.0,NaN,0.0,5.0,NaN,290.0,94107
2,03/09/2014,76.0,69.0,61.0,57.0,56.0,55.0,84.0,69.0,53.0,...,10.0,10.0,21.0,8.0,24.0,0.0,4.0,NaN,276.0,94107
3,04/09/2014,74.0,68.0,61.0,57.0,57.0,56.0,84.0,71.0,57.0,...,10.0,8.0,22.0,8.0,25.0,0.0,5.0,NaN,301.0,94107
4,05/09/2014,72.0,66.0,60.0,57.0,56.0,54.0,84.0,71.0,57.0,...,9.0,7.0,18.0,8.0,32.0,0.0,4.0,NaN,309.0,94107


### Notes
1. Date is primary key 
2. Zipcode is a foriegn key from the lat and longitudes

## Prelim Clean Up

In [8]:
## Removes the spaces from the titles for SQLite error      
station_data = station_data.rename(columns=lambda name: name.replace(" ","_"))
trip_data = trip_data.rename(columns=lambda name: name.replace(" ","_"))
weather_data = weather_data.rename(columns=lambda name: name.replace(" ","_"))

## Data into SQLite db

In [9]:
station_data.to_sql('station_data', db, if_exists='replace', index = False)
trip_data.to_sql('trip_data', db, if_exists='replace', index = False)
weather_data.to_sql('weather_data', db, if_exists='replace', index = False)

In [10]:
query='SELECT name FROM sqlite_master;'
run_query(query)

,name
0,station_data
1,trip_data
2,weather_data


In [17]:
## Geopy will turn lat and longs into cities which will be assiagned a zip[ code in weather.
def get_zipcode(df, geolocator, lat_field, lon_field):
    location = geolocator.reverse((df[lat_field], df[lon_field]))
    
    try:
        return location.raw['address']['city']
    except KeyError:
        return location.raw
    
geolocator = geopy.Nominatim(user_agent='my-bike-application')

In [18]:
query = '''
SELECT DISTINCT Lat, Long FROM station_data;
'''

distinct_map_points = run_query(query)


In [19]:
## kind long runn it once
zipcodes = distinct_map_points.apply(get_zipcode, axis=1, geolocator=geolocator, lat_field='Lat', lon_field='Long')
zipcodes

0          San Jose
1          San Jose
2          San Jose
3          San Jose
4          San Jose
5          San Jose
6          San Jose
7          San Jose
8          San Jose
9          San Jose
10         San Jose
11         San Jose
12         San Jose
13         San Jose
14     Redwood City
15     Redwood City
16     Redwood City
17     Redwood City
18     Redwood City
19     Redwood City
20     Redwood City
21     Redwood City
22    Mountain View
23    Mountain View
24    Mountain View
25    Mountain View
26    Mountain View
27    Mountain View
28    Mountain View
29        Palo Alto
          ...      
46    San Francisco
47    San Francisco
48    San Francisco
49    San Francisco
50    San Francisco
51    San Francisco
52    San Francisco
53    San Francisco
54    San Francisco
55    San Francisco
56    San Francisco
57    San Francisco
58    San Francisco
59    San Francisco
60    San Francisco
61    San Francisco
62    San Francisco
63    San Francisco
64    San Francisco


In [20]:
## One zip code doesnt show up, im going to pluge the lat and long into google to get it 
## it is 95192 
zipcodes.unique()


array(['San Jose', 'Redwood City', 'Mountain View', 'Palo Alto',
       'San Francisco'], dtype=object)

In [51]:
for code in zipcodes:
    if len(code) > 5:
        code= code[:5]
        
    print(code)


95110
95110
95113
95110
95113
95112
95192
95112
95192
95116
95112
95113
95110
95192
94063
94063
94063
94063
94063
94063
94603
94063
94039
94041
94040
94041
94040
94039
94040
94301
94301
94036
94301
94306
94111
94111
94111
94113
94104
94111
94105
94105
94111
94105
94102
94017
94105
94111
94103
94102
94102
94113
94017
94105
94105
94017
90103
94102
94102
94104
94107
94017
94017
94107
94102
94103
94102
94113
94111
94105
94104
94104
95112
94111
94063
95113


In [52]:
## San Francisco, San Mateo (Redwood city), Palo Alto, Mountain View(MV and ), San Jose
query = '''
SELECT DISTINCT zip FROM weather_data;
'''

run_query(query)

,Zip
0,94107
1,94063
2,94301
3,94041
4,95113
